### 5.learning to rank

&emsp;&emsp;参考SCScore 模型，虽然最终给出的一个分子的分数，但是在训练的数据是pair，损失函数是hinge_loss，但是提供了一个新的思路，就是pair的顺序问题。

&emsp;&emsp;有几种rank 的方法如下:

- 1.Pointwise Approach: 对单点打分，可以转换为回归和分类问题
- 2.Pariwise Approach: 将pair的数据打分，转换为回归和分类问题
- 3.Listwise Approach: 将完整的排序列表作为样本，转换为回归和分类问题

&emsp;&emsp;分析这三种rank 方法，选出哪种rank方法适合SCScore中的假设

- 1. Pointwise Approach: 对一个对象打分，没有准确的label ，然后并未考虑到序列的顺序关系
- 3. Listwise Approach: 考虑序列的整体顺序关系，但是在我们的训练数据中没有这种样本

&emsp;&emsp;因此基于 reactants < products 这种假设，可能适用的rank 方法是Pairwise Approach 方法，接下来介绍看的几篇典型文献。

#### 5.1 SVM Rank

&emsp;&emsp;可以学习一个分类器，来对对象对的排序进行分类并将分类器应用在排序任务中，将同一pair中的特征向量之间的差作为新的特征向量对待，在SVM训练的过程中，可以忽略掉负样本，因为是冗余的。

&emsp;&emsp;训练数据格式 ${((x_i^1,x_i^2),y_i)}$

- i = 1... m ，表示样本数量
- 每个样本包括两个特征向量$(x_i^1.x_i^2)$ 和$y_i$,表示哪个特征向量应该排在前面。

&emsp;&emsp;损失函数如下:

$$min_{w,\epsilon}\frac{1}{2}|w|^2 + C \sum_{i=1}^{m}\epsilon_i \tag{1}$$

$$s.t. y_i(w,x_i^1-x_i^2) >= 1 - \epsilon_i \tag{2}$$

$$\epsilon_i >= 0   i=1...m \tag{3}$$

&emsp;&emsp;等价于下面这个无约束优化问题，回归hinge loss函数的最小化

$$min_{w} \sum_{i=1}^{m}[1 - y_i(w,x_i^1 - x_i^2)]_{+} + \frac{1}{2C}|w|^2 \tag{4}$$

#### 5.2 RankNet

&emsp;&emsp;排序的好坏完全取决于模型的输出，而模型又由参数决定，因为问题转换为如何利用带label的训练数据去获得最优的模型参数w。

&emsp;&emsp;cost function 是RankNet算法的核心，能不能通过cross entropy的方式进行训练？

##### 1.相关性概率
1. 预测相关性概率：对于任意一个pair$(x_i,x_j)$，模型输出score 分别为$s_i,s_j$,那么根据模型的预测，$x_i$比$x_j$更难合成的概率为:

$$P_{i,j} = P(x_i > x_j) = \frac{1}{1 + e^(-sigmoid(s_i - s_j))} \tag{5}$$

2. 真实相关性概率:

$$P_{i,j}^{True} = \frac{1}{2}(1+S_{i,j})$$

如果$X_i$比$X_j$更难合成，那么$S_{i,j}$ = 1,否则 $S_{i,j}$ = -1

##### 2.代价函数
&emsp;&emsp;RankNet 从各个pair的相对关系来评价排序结果的好坏，排序的效果越好，那么有错误相关关系的pair就越少，对于每一个pair,使用交叉熵来度量其预测代价。

$$C_{i,j} = -P_{i,j}^{true}log(P_{i,j}) - (1-P_{i,j}^{True})log(1-P_{i,j})$$